<a href="https://colab.research.google.com/github/abi672003/GRU-Part-of-the-Toxic-Comment-Classification/blob/main/Mini_Project_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **GRU**

In [ ]:
!pip install seaborn

In [ ]:
!pip install plotly

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Embedding,BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:
train = pd.read_csv('/content/validation.csv')
validation = pd.read_csv('/content/validation.csv')
test = pd.read_csv('/content/test.csv')

In [ ]:
train.drop(['id','lang','toxic'],axis=1,inplace=True)

In [ ]:
train = train.loc[:12000,:]
train.shape

(8000, 1)

In [ ]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

211

In [ ]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''

    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, validation.toxic.values,
                                                  stratify=validation.toxic.values,
                                                  random_state=42,
                                                  test_size=0.2, shuffle=True)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 1500
token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
#zero pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)
word_index = token.word_index

In [ ]:
scores_model = []

In [ ]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 69601/69601 [00:00<00:00, 2085719.66it/s]


In [ ]:
%%time
with strategy.scope():
    # GRU with glove embeddings and two dense layers
     model = Sequential()
     model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
     model.add(SpatialDropout1D(0.3))
     model.add(GRU(300))
     model.add(Dense(1, activation='sigmoid'))

     model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         20880600  
                                                                 
 spatial_dropout1d (SpatialD  (None, 1500, 300)        0         
 ropout1D)                                                       
                                                                 
 gru (GRU)                   (None, 300)               541800    
                                                                 
 dense (Dense)               (None, 1)                 301       
                                                                 
Total params: 21,422,701
Trainable params: 542,101
Non-trainable params: 20,880,600
_________________________________________________________________
CPU times: user 548 ms, sys: 148 ms, total: 696 ms
Wall time: 634 ms


In [ ]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
100/100 [==============================] - 478s 5s/step - loss: 0.4933 - accuracy: 0.8462
Epoch 2/5
100/100 [==============================] - 460s 5s/step - loss: 0.4301 - accuracy: 0.8462
Epoch 3/5
100/100 [==============================] - 457s 5s/step - loss: 0.4297 - accuracy: 0.8462
Epoch 4/5
100/100 [==============================] - 457s 5s/step - loss: 0.4299 - accuracy: 0.8462
Epoch 5/5
100/100 [==============================] - 456s 5s/step - loss: 0.4305 - accuracy: 0.8462


In [ ]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

50/50 [==============================] - 62s 1s/step
Auc: 0.50%


In [ ]:
scores_model.append({'Model': 'GRU','AUC_Score': roc_auc(scores,yvalid)})
scores_model

[{'Model': 'GRU', 'AUC_Score': 0.5}]